In [3]:
# 경고 메시지를 표시하지 않으려면 아래의 2줄을 코딩하면 된다.
import warnings
warnings.filterwarnings('ignore') # 다시 경고 메시지를 보고싶다면 warnings.filterwarnings('default')를 실행하면 출력된다.

In [7]:
import requests # 타겟 사이트에 접속해서 크롤링 정보를 얻어온다.
import folium   # 지도를 표시하고 지도에 마커를 찍어준다.
import json     # 텍스트로 된 json 형태의 데이터를 파이썬의 딕셔너리 형태로 변환한다.
from pandas.io.json import json_normalize # 파이썬의 딕셔너리 형태의 데이터를 판다스 데이터프레임 형태로 변환한다.

In [8]:
# 전국의 특정 지역 스타벅스 매장의 위치을 찾아서 지도 위에 표시하기
# 1. requests로 스타벅스 매장 위치 데이터를 가져와서 딕셔너리 타입으로 변환한다.
# 2. json_normalize() 메소드로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터 프레임으로 변환한다.
# 3. folium 모듈을 사용해서 지도 위에 마커를 표시한다.

In [31]:
# 서울 스타벅스 전지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X49A85FZMZ'
req = requests.post(targetSite, data = {
    'p_sido_cd': '01',      # 시도 코드
    'p_gugun_cd': '',       # 구군 코드
    'ins_lat': 37.5108295,  # 위도
    'ins_lng': 127.0292880, # 경도
    'in_biz_cd': '',
    'set_date': '',
    'iend': 1500            # 크롤링해올 최대 매장의 개수
})
print(type(req.text))       # <class 'str'> => 딕셔너리로 변환한다.
print(req.text)

<class 'str'>
{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1509","s_name":"역삼아레나빌딩","tel":"1522-3232","fax":"02-568-3763","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 역삼동 721-13 아레나빌딩","park_info":null,"new_state":null,"theme_state":"T04@T05@T08@T16@T17@T20@T21@T30@T36@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":"1.63","all_

In [32]:
# json 모듈의 loads() 메소드로 스타벅스 매장 위치 정보가 저장된 json 타입의 문자열 데이터를 파이썬에서 처리학 위해서
# 딕셔너리 타입으로 변환한다.
star_json = json.loads(req.text)
print(type(star_json))
print(star_json)

<class 'dict'>
{'list': [{'seq': 0, 'sido_cd': None, 'sido_nm': None, 'gugun_cd': None, 'gugun_nm': None, 'code_order': None, 'view_yn': None, 'store_num': None, 'sido': None, 'gugun': None, 'address': None, 'new_img_nm': None, 'p_pro_seq': 0, 'p_view_yn': None, 'p_sido_cd': '', 'p_gugun_cd': '', 'p_store_nm': None, 'p_theme_cd': None, 'p_wireless_yn': None, 'p_smoking_yn': None, 'p_book_yn': None, 'p_music_yn': None, 'p_terrace_yn': None, 'p_table_yn': None, 'p_takeout_yn': None, 'p_parking_yn': None, 'p_dollar_assent': None, 'p_card_recharge': None, 'p_subway_yn': None, 'stb_store_file_renew': None, 'stb_store_theme_renew': None, 'stb_store_time_renew': None, 'stb_store_lsm': None, 's_code': '1509', 's_name': '역삼아레나빌딩', 'tel': '1522-3232', 'fax': '02-568-3763', 'sido_code': '01', 'sido_name': '서울', 'gugun_code': '0101', 'gugun_name': '강남구', 'addr': '서울특별시 강남구 역삼동 721-13 아레나빌딩', 'park_info': None, 'new_state': None, 'theme_state': 'T04@T05@T08@T16@T17@T20@T21@T30@T36@P80@P90', 'new_bo

In [33]:
# pandas.io.json 모듈의 json_normalize() 메소드로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
star_df = json_normalize(star_json, 'list')
print(type(star_df))
print(len(star_df))    # 행(row)의 개수, 서울에는 2020년 10월 8일 현재 526개의 매장이 있다.
print(star_df.columns) # 열(column)의 개수는 122개 => 너무 많다 => folium 작업에 필요한 column만 별도로 추출해서 사용한다.

<class 'pandas.core.frame.DataFrame'>
526
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p30', 'p70', 'p40', 'p80', 't22', 't21', 'p90', 't05', 't30', 't36'],
      dtype='object', length=122)


In [34]:
star_df

seq sido_cd sido_nm gugun_cd gugun_nm code_order view_yn store_num  sido  \
0      0    None    None     None     None       None    None      None  None   
1      0    None    None     None     None       None    None      None  None   
2      0    None    None     None     None       None    None      None  None   
3      0    None    None     None     None       None    None      None  None   
4      0    None    None     None     None       None    None      None  None   
..   ...     ...     ...      ...      ...        ...     ...       ...   ...   
521    0    None    None     None     None       None    None      None  None   
522    0    None    None     None     None       None    None      None  None   
523    0    None    None     None     None       None    None      None  None   
524    0    None    None     None     None       None    None      None  None   
525    0    None    None     None     None       None    None      None  None   

    gugun  ... p30 p70  p40 p80 t22 t21 p90 t05 t30 t36  
0    None  ...   0   0    0   0   0   0   0   0   0   0  
1    None  ...   0   0    0   0   0   0   0   0   0   0  
2    None  ...   0   0    0   0   0   0   0   0   0   0  
3    None  ...   0   0    0   0   0   0   0   0   0   0  
4    None  ...   0   0    0   0   0   0   0   0   0   0  
..    ...  ...  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  
521  None  ...   0   0    0   0   0   0   0   0   0   0  
522  None  ...   0   0    0   0   0   0   0   0   0   0  
523  None  ...   0   0    0   0   0   0   0   0   0   0  
524  None  ...   0   0    0   0   0   0   0   0   0   0  
525  None  ...   0   0    0   0   0   0   0   0   0   0  

[526 rows x 122 columns]

In [35]:
# 주요한 column 몇 가지를 선택해서 지도에 marker를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name       => 지점 이름
# sido_code    => 시도 코드
# sido_name    => 시도 이름
# gugun_code   => 구군 코드
# gugun_name   => 구군 이름
# doro_address => 도로명 주소
# lat          => 위도
# lot          => 경도
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map

s_name sido_code sido_name gugun_code gugun_name  \
0    역삼아레나빌딩        01        서울       0101        강남구   
1     논현역사거리        01        서울       0101        강남구   
2    신사역성일빌딩        01        서울       0101        강남구   
3     국기원사거리        01        서울       0101        강남구   
4     스탈릿대치R        01        서울       0101        강남구   
..       ...       ...       ...        ...        ...   
521     중랑구청        01        서울       0125        중랑구   
522     사가정역        01        서울       0125        중랑구   
523      상봉역        01        서울       0125        중랑구   
524    묵동이마트        01        서울       0125        중랑구   
525       묵동        01        서울       0125        중랑구   

                          doro_address          lat          lot  
0              서울특별시 강남구 언주로 425 (역삼동)    37.501087   127.043069  
1             서울특별시 강남구 강남대로 538 (논현동)    37.510178   127.022223  
2             서울특별시 강남구 강남대로 584 (논현동)    37.514132   127.020563  
3             서울특별시 강남구 테헤란로 125 (역삼동)    37.499517   127.031495  
4           서울특별시 강남구 남부순환로 2947 (대치동)    37.494668   127.062583  
..                                 ...          ...          ...  
521                   서울특별시 중랑구 신내로 72  37.60538908  127.0957558  
522                  서울특별시 중랑구 면목로 310    37.579594   127.087966  
523    서울특별시 중랑구 망우로 307, ,3,4번지 (상봉동)     37.59689    127.08647  
524  서울특별시 중랑구 동일로 932, 묵동이마트 B1층 (묵동)    37.613433   127.077484  
525                 서울특별시 중랑구 동일로 952     37.615368   127.076633  

[526 rows x 8 columns]

In [36]:
# dtypes를 이용해 판다스 데이터프레임을 구성하는 column들의 자료형을 확인할 수 있다.
star_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [38]:
# info() 메소드를 사용해도 판다스 데이터프레임을 구성하는 column들의 자료형을 확인할 수 있다.
star_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        526 non-null    object
 1   sido_code     526 non-null    object
 2   sido_name     526 non-null    object
 3   gugun_code    526 non-null    object
 4   gugun_name    526 non-null    object
 5   doro_address  526 non-null    object
 6   lat           526 non-null    object
 7   lot           526 non-null    object
dtypes: object(8)
memory usage: 33.0+ KB


In [39]:
# astype() 메소드로 위도, 경도 데이터 타입을 object(판다스에서 문자열을 의미한다.)에서 float(실수)로 변환한다.
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
star_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [40]:
# 종각점
star_df_map[star_df_map['s_name'] == '종각']

s_name sido_code sido_name gugun_code gugun_name            doro_address  \
390     종각        01        서울       0118        종로구  서울특별시 종로구 종로 64 (종로2가)   

           lat         lot  
390  37.569918  126.984528

In [41]:
# 광교점
star_df_map[star_df_map['s_name'] == '광교']

s_name sido_code sido_name gugun_code gugun_name             doro_address  \
391     광교        01        서울       0118        종로구  서울특별시 종로구 청계천로 59 (관철동)   

           lat         lot  
391  37.568928  126.984216

In [46]:
# folium의 Map() 메소드로 location 속성으로 지정한 위치를 중심으로 하는 zoom_start 만큼의 배율을 사용하는 지도를 만든다.
star_map = folium.Map(location = [37.5662994, 126.9757564], zoom_start = 16) # 서울특별시청을 중심으로 하는 지도
# folium의 Marker() 메소드로 location 속성으로 지정한 위치에 표시할 마커를 만들어 add_to() 메소드로 지도에 추가한다.
# Marker() 메소드에 popup 속성으로 마커를 클릭했을 때 표시할 팝업을 만들 수 있다. => jupyter notebook에서 한글이 깨져보인다.
folium.Marker(location = [37.569918, 126.984528], popup = '종각점').add_to(star_map) # 종각점
folium.Marker(location = [37.568928, 126.984216], popup = '광교점').add_to(star_map) # 광교점
# save() 메소드로 folium을 사용해 만든 지도를 HTML 파일로 저장할 수 있다. => popup으로 지정한 한글이 깨지지 않는다.
star_map.save('./star1.html')
star_map # 지도를 화면에 표시한다.

In [51]:
# 서울시 스타벅스 전 지점을 지도 위에 표시한다.
# 지도를 작성할 때 지도의 중심으로 설정할 특정 위치가 없다면 지도위에 표시할 모든 지점의 위도와 경도의 평균을 중심으로 하는
# 지도를 만들면 된다.
# 판다스 데이터프레임에 저장된 데이터의 평균은 mean() 메소드를 사용해서 계산한다.
star_map = folium.Map(location = [star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start = 12)
# iterrows() 메소드는 판다스 데이터프레임의 인덱스 번호와 데이터프레임의 데이터를 한 행씩 리턴시킨다.
for i, row in star_df_map.iterrows():
    # print(i, row)
    popup = '{} - {}'.format(row['s_name'], row['doro_address'])
    folium.Marker(location = [row['lat'], row['lot']], popup = popup).add_to(star_map)
star_map.save('./star2.html')
star_map

In [55]:
# 강남구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X49A85FZMZ'
req = requests.post(targetSite, data = {
    'p_sido_cd': '',
    'p_gugun_cd': '0101',
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292880,
    'in_biz_cd': '',
    'set_date': '',
    'iend': 1500
})
star_json = json.loads(req.text)
star_df = json_normalize(star_json, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
# star_df_map

In [54]:
star_map = folium.Map(location = [star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start = 14)
for i, row in star_df_map.iterrows():
    popup = '{} - {}'.format(row['s_name'], row['doro_address'])
    folium.Marker(location = [row['lat'], row['lot']], popup = popup).add_to(star_map)
star_map.save('./star3.html')
star_map

In [56]:
# 경기도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X49A85FZMZ'
req = requests.post(targetSite, data = {
    'p_sido_cd': '08',
    'p_gugun_cd': '',
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292880,
    'in_biz_cd': '',
    'set_date': '',
    'iend': 1500
})
star_json = json.loads(req.text)
star_df = json_normalize(star_json, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
# star_df_map

In [57]:
star_map = folium.Map(location = [star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start = 10)
for i, row in star_df_map.iterrows():
    popup = '{} - {}'.format(row['s_name'], row['doro_address'])
    folium.Marker(location = [row['lat'], row['lot']], popup = popup).add_to(star_map)
star_map.save('./star4.html')
star_map

In [58]:
# 제주도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=X49A85FZMZ'
req = requests.post(targetSite, data = {
    'p_sido_cd': '16',
    'p_gugun_cd': '',
    'ins_lat': 37.5108295,
    'ins_lng': 127.0292880,
    'in_biz_cd': '',
    'set_date': '',
    'iend': 1500
})
star_json = json.loads(req.text)
star_df = json_normalize(star_json, 'list')
star_df_map = star_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
star_df_map['lat'] = star_df_map['lat'].astype(float)
star_df_map['lot'] = star_df_map['lot'].astype(float)
# star_df_map

In [61]:
star_map = folium.Map(location = [star_df_map['lat'].mean(), star_df_map['lot'].mean()], zoom_start = 11)
for i, row in star_df_map.iterrows():
    popup = '{} - {}'.format(row['s_name'], row['doro_address'])
    folium.Marker(location = [row['lat'], row['lot']], popup = popup).add_to(star_map)
star_map.save('./star5.html')
star_map